In [38]:
from pyspark import SparkFiles
from pyspark import SparkContext 
from pyspark import SQLContext
import sys
import os

In [5]:
sc = SparkContext(appName ='First Lesson', master = 'local')
sc

<SparkContext master=local appName=First Lesson>

In [6]:
rdd = sc.textFile('file:///Users/tjmask/Desktop/test.txt')

In [7]:
rdd.take(2)

['USE db_countries;', '-- IMPORTING DATA FROM A CSV']

In [8]:
def fuc(lines):
    lines =lines.lower()
    lines = lines.split()
    return lines

In [9]:
rdd1 = rdd.map(fuc)

In [10]:
rdd1.take(2)

[['use', 'db_countries;'], ['--', 'importing', 'data', 'from', 'a', 'csv']]

In [12]:
rdd2 = rdd.flatMap(fuc)
rdd2.take(5)

['use', 'db_countries;', '--', 'importing', 'data']

In [15]:
stopwords=['a', 'use', 'the', 'is']
rdd3 = rdd2.filter(lambda x: x not in stopwords)
rdd3.take(2)

['db_countries;', '--']

In [17]:
rdd4 = rdd3.groupBy(lambda x: x[0:3])
rdd4.take(5)

[('db_', <pyspark.resultiterable.ResultIterable at 0x114eb5fd0>),
 ('--', <pyspark.resultiterable.ResultIterable at 0x114eb5f90>),
 ('imp', <pyspark.resultiterable.ResultIterable at 0x114eb9250>),
 ('dat', <pyspark.resultiterable.ResultIterable at 0x114eb90d0>),
 ('fro', <pyspark.resultiterable.ResultIterable at 0x114eb93d0>)]

In [27]:
print([(k,list(v)) for (k, v) in rdd4.take(5)])

[('db_', ['db_countries;']), ('--', ['--', '--', '--']), ('imp', ['importing']), ('dat', ['data']), ('fro', ['from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from'])]


In [30]:
num_rdd = sc.parallelize(range(1,5))
num_rdd.reduce(lambda x,y: x*y)

24

In [41]:
sqlContext = SQLContext(sc)
sqlContext

In [46]:
## sparkconf
import pyspark
rmowers = sqlContext.read.csv('file:///Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/RidingMowers.csv', \
                              inferSchema='true', header=True)

In [47]:
rmowers.show()

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  60.0|    18.4|    Owner|
|  85.5|    16.8|    Owner|
|  64.8|    21.6|    Owner|
|  61.5|    20.8|    Owner|
|  87.0|    23.6|    Owner|
| 110.1|    19.2|    Owner|
| 108.0|    17.6|    Owner|
|  82.8|    22.4|    Owner|
|  69.0|    20.0|    Owner|
|  93.0|    20.8|    Owner|
|  51.0|    22.0|    Owner|
|  81.0|    20.0|    Owner|
|  75.0|    19.6| Nonowner|
|  52.8|    20.8| Nonowner|
|  64.8|    17.2| Nonowner|
|  43.2|    20.4| Nonowner|
|  84.0|    17.6| Nonowner|
|  49.2|    17.6| Nonowner|
|  59.4|    16.0| Nonowner|
|  66.0|    18.4| Nonowner|
+------+--------+---------+
only showing top 20 rows



In [48]:
rmowers.printSchema()

root
 |-- Income: double (nullable = true)
 |-- Lot_Size: double (nullable = true)
 |-- Ownership: string (nullable = true)



In [49]:
rmowers.count()

24

In [50]:
rmowers.select('Income', 'Ownership').show()

+------+---------+
|Income|Ownership|
+------+---------+
|  60.0|    Owner|
|  85.5|    Owner|
|  64.8|    Owner|
|  61.5|    Owner|
|  87.0|    Owner|
| 110.1|    Owner|
| 108.0|    Owner|
|  82.8|    Owner|
|  69.0|    Owner|
|  93.0|    Owner|
|  51.0|    Owner|
|  81.0|    Owner|
|  75.0| Nonowner|
|  52.8| Nonowner|
|  64.8| Nonowner|
|  43.2| Nonowner|
|  84.0| Nonowner|
|  49.2| Nonowner|
|  59.4| Nonowner|
|  66.0| Nonowner|
+------+---------+
only showing top 20 rows



In [51]:
rmowers.describe('Income').show()

+-------+------------------+
|summary|            Income|
+-------+------------------+
|  count|                24|
|   mean|           68.4375|
| stddev|19.793143575710648|
|    min|              33.0|
|    max|             110.1|
+-------+------------------+



In [52]:
rmowers.filter(rmowers.Income >= '90').show()

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
| 110.1|    19.2|    Owner|
| 108.0|    17.6|    Owner|
|  93.0|    20.8|    Owner|
+------+--------+---------+



In [60]:
rmowers.where((rmowers.Income >= 60) & (rmowers.Ownership=='Owner')).show()

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  60.0|    18.4|    Owner|
|  85.5|    16.8|    Owner|
|  64.8|    21.6|    Owner|
|  61.5|    20.8|    Owner|
|  87.0|    23.6|    Owner|
| 110.1|    19.2|    Owner|
| 108.0|    17.6|    Owner|
|  82.8|    22.4|    Owner|
|  69.0|    20.0|    Owner|
|  93.0|    20.8|    Owner|
|  81.0|    20.0|    Owner|
+------+--------+---------+



In [61]:
rmowers.registerTempTable('rmowers_df')

In [68]:
sqlContext.sql('select * from rmowers_df').show(5)

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  60.0|    18.4|    Owner|
|  85.5|    16.8|    Owner|
|  64.8|    21.6|    Owner|
|  61.5|    20.8|    Owner|
|  87.0|    23.6|    Owner|
+------+--------+---------+
only showing top 5 rows



In [70]:
sqlContext.sql('select min(Income) min_income from rmowers_df').show()

+----------+
|min_income|
+----------+
|      33.0|
+----------+



In [71]:
sqlContext.sql("select * from rmowers_df where Income in (select min(Income) min_income from rmowers_df)").show()

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  33.0|    18.8| Nonowner|
+------+--------+---------+



### MLlib

In [73]:
from pyspark import SparkContext, SparkConf

In [74]:
conf = SparkConf().setMaster('local[*]').setAppName('first-ML')

In [75]:
conf

In [76]:
import pandas as pd
import numpy as np

In [88]:
toyota = pd.read_csv('/Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/ToyotaCorolla.csv')

In [89]:
toyota.head(2)

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,0,0,0,1,0,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0


In [91]:
print(toyota.shape)

(1436, 39)


In [112]:
toyota_new = toyota.drop(columns =['Id', 'Model', 'Fuel_Type', 'Color']).dropna()
toyota_new.iloc[:, 34]

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      1
18      0
19      0
20      1
21      0
22      1
23      0
24      0
25      1
26      0
27      0
28      0
29      0
       ..
1406    0
1407    1
1408    0
1409    1
1410    0
1411    0
1412    0
1413    1
1414    1
1415    0
1416    0
1417    0
1418    0
1419    0
1420    0
1421    0
1422    0
1423    0
1424    0
1425    0
1426    1
1427    0
1428    0
1429    1
1430    0
1431    0
1432    0
1433    0
1434    0
1435    0
Name: Tow_Bar, Length: 1436, dtype: int64

In [105]:
toyota_new.to_csv('/Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/toyota_new.txt', sep=",", index=False, header=None)

In [106]:
points = sc.textFile('file:///Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/toyota_new.txt')

In [119]:
points.take(1)

['13500,23,10,2002,46986,90,1,0,2000,3,4,5,210,1165,0,1,3,1,1,1,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0']

In [123]:
def parse_point(line):
    values = [s for s in line.strip().split(',')]
    return LabeledPoint(values[34], values[0:33])

In [124]:
data_parsed = points.map(parse_point)
data_parsed.take(5)

[LabeledPoint(0.0, [13500.0,23.0,10.0,2002.0,46986.0,90.0,1.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,0.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [13750.0,23.0,10.0,2002.0,72937.0,90.0,1.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,0.0,1.0,3.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [13950.0,24.0,9.0,2002.0,41711.0,90.0,1.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [14950.0,26.0,7.0,2002.0,48000.0,90.0,0.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [13750.0,30.0,3.0,2002.0,38500.0,90.0,0.0,0.0,2000.0,3.0,4.0,5.0,210.0,1170.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0])]

In [125]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

In [126]:
train, test = data_parsed.randomSplit([0.7, 0.3])

In [128]:
train_model = DecisionTree.trainClassifier(train, numClasses=5, categoricalFeaturesInfo= {}, maxDepth=3)

In [129]:
predictions = train_model.predict(test.map(lambda x: x.features))

In [196]:
predictions.take(5)

[0.0, 0.0, 0.0, 0.0, 0.0]

In [147]:
LabelsAndPrediction = test.map(lambda x: x.label).zip(predictions)
LabelsAndPrediction.take(10)

[(0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (1.0, 0.0),
 (0.0, 0.0)]

In [249]:
# testErr = LabelsAndPrediction.map(lambda x: x.keys()=x.values()).count()/(predictions.count())
# # testErr

In [258]:
keys = LabelsAndPrediction.keys().collect()
values = LabelsAndPrediction.values().collect()
count=0
for i in range(410):
    if keys[i] == values[i]:
        count += 1
count

295

In [260]:
testErr = 1 - count/(predictions.count())
testErr

0.2804878048780488

In [163]:
print(train_model.toDebugString())

DecisionTreeModel classifier of depth 3 with 7 nodes
  If (feature 1 <= 31.5)
   If (feature 4 <= 33403.0)
    Predict: 0.0
   Else (feature 4 > 33403.0)
    If (feature 27 <= 0.5)
     Predict: 0.0
    Else (feature 27 > 0.5)
     Predict: 1.0
  Else (feature 1 > 31.5)
   Predict: 0.0



In [176]:
LabelsAndPrediction.collect()[0]

(0.0, 0.0)

In [210]:
data_key = sc.parallelize([('a', 1),('b', 3),('c', 2),('a', 1),('d', 2),('b', 1),('d', 3)],4)
# data_key.reduceByKey(lambda x, y: x + y).collect()
data_key.collect()

[('a', 1), ('b', 3), ('c', 2), ('a', 1), ('d', 2), ('b', 1), ('d', 3)]

In [211]:
data_key.reduceByKey(lambda x, y: x==y).collect()

[('b', False), ('c', 2), ('a', True), ('d', False)]

In [ ]:
data_key.

In [216]:
LabelsAndPrediction.filter(lambda x, y: x==y)

PythonRDD[281] at RDD at PythonRDD.scala:53

In [218]:
type(LabelsAndPrediction)

pyspark.rdd.RDD